# Taxi Travel in New York City during the Christmas Holidays: An Analysis of Destinations, Trends, and Factors

In [ ]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from pmdarima import auto_arima
from IPython.display import HTML
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace import sarimax
import pickle
%matplotlib inline

## 数据集来源及获取
本文主要用到了如下数据集：
1. 纽约市2017年-2021每年12月的黄色出租车和绿色出租车数据 - [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
2. 对应数据区域的shapefile - [TLC Trip Record Data](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)
3. 对应时间的天气数据（API获取） - [OpenWeather](https://openweathermap.org/api/one-call-3)

### 天气数据获取

In [ ]:
# 插入获取天气数据代码

### 载入区域地理数据

In [ ]:
nyc_zones_geo = gpd.read_file('../data/taxi_zones/taxi_zones.shp')
#transform the coordinate system to WGS84
nyc_zones_geo = nyc_zones_geo.to_crs(epsg=4326)

### 出租车行程数据处理
这一节旨在将出租车形成数据集计为小时车流数据，集计后的数据包括：
1. 日期和小时
2. 出发区域ID
3. 到达区域ID
4. 行程数量
5. 行程人数
6. 旅行费用
7. 载客系数（行程人数/行程数量）

In [ ]:
# 插入数据集计代码

## Find the hot points
* calculate the in-and-out volumn for each zone by hours
* give the zones a rank, transfer the rank to point
* add all time points, find the hotest points.

### Get the score of each point per hour.   
Because of the time complexity of bellow function is too high, I used AWS to calculate the traffic in each zone.

In [ ]:
# get the score for each zone by day and hours
def get_in_out_volumn_score(df, day_list, hour_list, year):
    """
    Calculate the in-and-out volumn for each zone by day and hours, and give each zone a score based on the volumn.
    Input:
        df: a dataframe containing the counted data
        day_list: a list of days
        hour_list: a list of hours
    Output:
        df_in_out_volumn: a dataframe containing the in-and-out volumn for each zone by day and hours, and the score
    """
    # calculate the in-and-out volumn for each zone by day and hours
    ## get the location id
    location_id_list = list(range(1,264))
    ## get the day and hour
    day_list = day_list
    hour_list = hour_list
    ## get the in-and-out volumn for each zone by day and hours, and store them in a dictionary
    in_out_volumn = {}
    for location_id in location_id_list:
        for day in day_list:
            for hour in hour_list:
                in_out_volumn[(location_id, day, hour)] = [df.loc[(df['DOLocationID']==location_id) & (df['day']==day) & (df['hour']==hour), 'trip_count'].sum(), df.loc[(df['PULocationID']==location_id) & (df['day']==day) & (df['hour']==hour), 'trip_count'].sum()]
        print(f'Finish calculating in-and-out volumn for zone {location_id} of {year}.')
    # convert the dictionary to a dataframe
    df_in_out_volumn = pd.DataFrame.from_dict(in_out_volumn, orient='index', columns=['in_volumn', 'out_volumn'])
    # add day hour and location id as columns
    df_in_out_volumn['day'] = df_in_out_volumn.index.map(lambda x: x[1])
    df_in_out_volumn['hour'] = df_in_out_volumn.index.map(lambda x: x[2])
    df_in_out_volumn['location_id'] = df_in_out_volumn.index.map(lambda x: x[0])
    # add the total volumn
    df_in_out_volumn['total_volumn'] = df_in_out_volumn['in_volumn'] + df_in_out_volumn['out_volumn']
    # reorder the columns
    df_in_out_volumn = df_in_out_volumn[['day', 'hour', 'location_id', 'in_volumn', 'out_volumn', 'total_volumn']]
    # reset the index
    df_in_out_volumn = df_in_out_volumn.reset_index(drop=True)

    # calculate the rank of the total volumn for each zone by day and hours
    df_in_out_volumn['total_volumn_rank'] = df_in_out_volumn.groupby(['day', 'hour'])['total_volumn'].rank(ascending=False)
    # give the zone a score = 1/rank * number of zones
    df_in_out_volumn['total_volumn_score'] = (1/df_in_out_volumn['total_volumn_rank']) * 263
    
    return df_in_out_volumn

def plot_in_out_volumn_score(df_in_out_volumn, day_list, hour_list, year):
    """
    Plot the score for each zone by day and hours.
    Input:
        df_in_out_volumn: a dataframe containing the in-and-out volumn for each zone by day and hours, and the score
        day_list: a list of days
        hour_list: a list of hours
    Output:
        None
    """
    # plot the score for each zone by day and hours
    ## get the day and hour
    day_list = day_list
    hour_list = hour_list
    ## plot the score for each zone by day and hours
    for day in day_list:
        for hour in hour_list:
            plt.figure(figsize=(20,10))
            sns.barplot(x='location_id', y='total_volumn_score', data=df_in_out_volumn.loc[(df_in_out_volumn['day']==day) & (df_in_out_volumn['hour']==hour), :])
            plt.title('day: {}, hour: {}'.format(day, hour))
            plt.xticks([])
            plt.savefig(f'../data/figures/in_out_volumn_score{year}_{day}_{hour}.png')
            plt.close()

def get_hot_score(year_list, day_list, hour_list):
    for year in year_list:
        # read the data
        df = pd.read_csv(f'../data/processed_nyc_data/countdata_{year}-12.csv')
        # calculate the in-and-out volumn for each zone by day and hours, and give each zone a score based on the volumn
        df_in_out_volumn = get_in_out_volumn_score(df, day_list, hour_list,year)
        df_in_out_volumn.to_csv(f'../data/processed_nyc_data/in_out_volumn_score_{year}-12.csv', index=False)
        print(f'Finish getting score for {year}-12.')
        # plot the score for each zone by day and hours
        plot_in_out_volumn_score(df_in_out_volumn, day_list, hour_list, year)


In [ ]:
# # These codes run in AWS
# year_list = range(2017, 2022)  # 2017-2021
# day_list = range(1, 32)  # 1-31
# hour_list = range(0, 24)  # 0-23
# get_hot_score(year_list, day_list, hour_list)

### find the hotest points for each year

In [ ]:
def get_ranks(year):
    df = pd.read_csv(f'../data/processed_nyc_data/{year}.csv')
    df['year_score'] = df.groupby('location_id')['total_volumn_score'].transform('sum')
    df['year_volumn'] = df.groupby('location_id')['total_volumn'].transform('sum')
    df_hot = df[['location_id', 'year_score','year_volumn']].drop_duplicates()

    df_hot['volumn_rank'] = df_hot['year_volumn'].rank(ascending=False)
    df_hot['score_rank'] = df_hot['year_score'].rank(ascending=False)
    
    df_hot['fluctuation_coef'] = df_hot['score_rank'] / df_hot['volumn_rank']
    df_hot['fluctuation_rank'] = df_hot['fluctuation_coef'].rank(ascending=False)
    
    df_hot = df_hot.sort_values(by='year_score', ascending=False)
    df_hot = df_hot[['location_id', 'year_score', 'score_rank', 'year_volumn', 'volumn_rank', 'fluctuation_coef', 'fluctuation_rank']]

    return df_hot

def plot_hot_zone(df_hot, df_geo, year):
    # merge the year score to the geodata
    df_hot_geo = df_geo.merge(df_hot, left_on='LocationID', right_on='location_id', how='right')
    plt.figure(figsize=(20,10))
    df_hot_geo.plot(column='year_score', cmap='YlOrRd', edgecolor='black', legend=True)
    plt.savefig(f'../data/figures/hot_zone_{year}.png')
    plt.close()
    return None

def get_zones_rank(year_list):
    df_hots = {}
    for year in year_list:
        # get the 16 hotest zone for each year
        df_hot = get_ranks(year)
        df_hots[year] = df_hot

    return df_hots

In [ ]:
hot_zones = get_zones_rank(range(2017, 2022))

In [ ]:
# plot the first 30 hot zone's fluctuation rank on the map
def plot_hot_zone(dict_hot, df_geo, year):
    # merge the year score to the geodata
    df_hot_geo = df_geo.merge(dict_hot[year], left_on='LocationID', right_on='location_id', how='right')

    # filter the hot zone which volumn_rank is less than 31
    df_hot_geo = df_hot_geo.loc[df_hot_geo['volumn_rank']<=30, :]

    # add center point coordinates to the dataframe
    df_hot_geo['center_lat'] = df_hot_geo['geometry'].centroid.y
    df_hot_geo['center_lon'] = df_hot_geo['geometry'].centroid.x


    threshold_scale = np.linspace(df_hot_geo['fluctuation_rank'].min(),
                              df_hot_geo['fluctuation_rank'].max(), 4, dtype=float).tolist()

    
    # plot the hot zone with base map in folium
    m = folium.Map(location=[40.75, -73.9], zoom_start=11, tiles='Stamen Toner')
    folium.Choropleth(
        geo_data=df_hot_geo,
        name='choropleth',
        data=df_hot_geo,
        columns=['location_id', 'fluctuation_rank'],
        key_on='feature.properties.location_id',
        fill_color='RdYlBu',
        fill_opacity=0.7,
        line_opacity=1.0,
        threshold_scale=threshold_scale,
        legend_name='fluctuation').add_to(m)
    # always show zone id to the map as a label 
    folium.features.GeoJson(
        df_hot_geo,
        style_function=lambda feature: {
            'fillColor': 'transparent',
            'color': 'transparent',
            'weight': 0,
            'dashArray': '5, 5'
        },
        highlight_function=lambda x: {'weight':0.1, 'color':'black'},
        tooltip=folium.features.GeoJsonTooltip(
            fields=['location_id'],
            aliases=['Zone ID:'],
            localize=True,
            sticky=False
        )
    ).add_to(m)

    folium.LayerControl().add_to(m)
    m.save(f'../data/figures/hot_zone_{year}.html')
    # display(m)
    return None


plot_hot_zone(hot_zones, nyc_zones_geo, 2017)
plot_hot_zone(hot_zones, nyc_zones_geo, 2018)
plot_hot_zone(hot_zones, nyc_zones_geo, 2019)
plot_hot_zone(hot_zones, nyc_zones_geo, 2020)
plot_hot_zone(hot_zones, nyc_zones_geo, 2021)

已经做了的：
1. 给所有区域以年为单位进行流量和热度排名,其中：
   1. 流量排名：以31天24小时流量和进行排名，这反映了其在这一个月总的流量
   2. 热度排名：以31天24小时热度分数和进行排名，这反映了其在一个月中成为热点区域的频率
2. 找到每年热点区域（用于后续时间序列预测）
3. 流量排名和热度排名的差距显示了一些地区车流量的波动性显著强于另一些地区

要做的：
1. 比较各热点区域历年排名变化（可视化）
2. 比较一天内热点变化（工作日，一般周末，圣诞假期）（可视化）
3. auto_arima

## 时间定性分析

概览：
1. 比较历年热点区域变化情况
2. 比较一天内热点变化趋势（工作日、周末、圣诞节）
3. 比较同一区域的流量排名和热度排名，分类如下：
   1. 流量排名/热度排名 > 1 ：时间波动性较强
   2. 流量排名/热度排名 < 1 : 时间波动性较弱
4. 分别可视化波动性较强和较弱的区域，观察其分布


## ARIMA

概览：
1. 确定用于ARIMA分析的10个区域
2. 生成这些区域间交通流量的时间序列（5\*10\*10）（包括天气信息）
3. 对500条时间序列进行ARIMA分解，获取其总体趋势和季节性趋势
4. 总结结果

In [ ]:
# find the 10 hostest zone in most years
## add 5 years' score together
df_hot_5years = pd.DataFrame()
df_hot_5years['location_id'] = hot_zones[2017]['location_id']
for year in range(2017, 2022):
    df_hot_5years[year] = hot_zones[year]['year_score']
df_hot_5years['total_score'] = df_hot_5years.sum(axis=1)
df_hot_5years = df_hot_5years.sort_values(by='total_score', ascending=False)
df_hot_5years = df_hot_5years.reset_index(drop=True)
df_hot_5years = df_hot_5years.loc[:9, :]
df_hot_5years = nyc_zones_geo.merge(df_hot_5years, left_on='LocationID', right_on='location_id', how='right')
df_hot_5years = df_hot_5years[['location_id', 'total_score', 'geometry','zone']]

## show a table of the 10 hostest zone in most years
df_hot_5years[['zone', 'total_score']]

In [ ]:
# create time series for each pair of zones, and save them to a dictionary
def create_time_series(year, location_id_list):
    # create a template time series dataframe
    count_df = pd.read_csv(f'../data/processed_nyc_data/countdata_{year}-12.csv')

    temp_time_series = pd.DataFrame()
    temp_time_series['date'] = np.repeat(np.arange(1, 32), 24)
    temp_time_series['hour'] = np.tile(np.arange(0, 24), 31)
    
    all_time_series = {}
    
    for location_id1 in location_id_list:
        for location_id2 in location_id_list:
            if location_id1 != location_id2:
                series_name = f'{location_id1}to{location_id2}'
                all_time_series[series_name] = temp_time_series.copy()
                temp_count = count_df.loc[(count_df['PULocationID']==location_id1) & (count_df['DOLocationID']==location_id2), :]
                all_time_series[series_name] = all_time_series[series_name].merge(temp_count, left_on=['date', 'hour'], right_on=['day', 'hour'], how='left')
                # fill the nan with 0
                all_time_series[series_name] = all_time_series[series_name].fillna(0)
                # keep the columns we need
                all_time_series[series_name] = all_time_series[series_name][['date','hour','trip_count']]
                # create a datetime column
                all_time_series[series_name]['datetime']  = pd.to_datetime(all_time_series[series_name]['date'].astype(str) + '-12-' + str(year) + ' ' + all_time_series[series_name]['hour'].astype(str) + ':00:00', format='%d-%m-%Y %H:%M:%S')
    return all_time_series



location_id_list = df_hot_5years['location_id'].tolist()

all_time_series_2017 = create_time_series(2017, location_id_list)
all_time_series_2018 = create_time_series(2018, location_id_list)
all_time_series_2019 = create_time_series(2019, location_id_list)
all_time_series_2020 = create_time_series(2020, location_id_list)
all_time_series_2021 = create_time_series(2021, location_id_list)

In [ ]:
# decompose the time series for each pair of zones
def decompose_time_series(time_series_dict):
    all_time_series_decomposed = {}
    for time_series_key in time_series_dict.keys():
        temp_series = time_series_dict[time_series_key]
        all_time_series_decomposed[time_series_key] = seasonal_decompose(temp_series['trip_count'], model='additive', period=168).trend
        time_series_dict[time_series_key] = temp_series
    return all_time_series_decomposed

#plot all time series in one figure
def plot_all_time_series_decomposed(all_time_series, index, year, file_name):
    fig, ax = plt.subplots(10, 10, figsize=(20, 20))
    n = 0
    for i in range(10):
        for j in range(10):
            if i != j:
                # plot the time series without x axis label and y axis label
                all_time_series[index[n]].plot(ax=ax[i, j], legend=False, xlabel=None, ylabel=None,xticks=[])
                ax[i, j].set_xlabel(f'{index[n]}')
                ax[i, j].set_title('')
                n += 1
    plt.savefig(f'../data/figures/{file_name}')
    plt.close()
    return None



all_time_series_2017_decomposed = decompose_time_series(all_time_series_2017)
all_time_series_2018_decomposed = decompose_time_series(all_time_series_2018)
all_time_series_2019_decomposed = decompose_time_series(all_time_series_2019)
all_time_series_2020_decomposed = decompose_time_series(all_time_series_2020)
all_time_series_2021_decomposed = decompose_time_series(all_time_series_2021)

plot_all_time_series_decomposed(all_time_series_2017_decomposed, index, 2017,'time_series_2017_decomposed.png')
plot_all_time_series_decomposed(all_time_series_2018_decomposed, index, 2018,'time_series_2018_decomposed.png')
plot_all_time_series_decomposed(all_time_series_2019_decomposed, index, 2019,'time_series_2019_decomposed.png')
plot_all_time_series_decomposed(all_time_series_2020_decomposed, index, 2020,'time_series_2020_decomposed.png')
plot_all_time_series_decomposed(all_time_series_2021_decomposed, index, 2021,'time_series_2021_decomposed.png')

In [ ]:

## get all index of the time series
index = list(key for key in all_time_series_2017.keys())


#plot all time series in one figure
def plot_all_time_series(all_time_series, index, year, file_name):
    fig, ax = plt.subplots(10, 10, figsize=(20, 20))
    n = 0
    for i in range(10):
        for j in range(10):
            if i != j:
                # plot the time series without x axis label and y axis label
                all_time_series[index[n]].plot(x='datetime', y='trip_count', ax=ax[i, j], legend=False, xlabel=None, ylabel=None,xticks=[])
                ax[i, j].set_xlabel(f'{index[n]}')
                ax[i, j].set_title('')
                n += 1
    plt.title(f'time series of taxi flow in {year}')
    plt.savefig(f'../data/figures/{file_name}')
    plt.close()
    return None

plot_all_time_series(all_time_series_2017, index, 2017, 'time_series_2017.png')
plot_all_time_series(all_time_series_2018, index, 2018, 'time_series_2018.png')
plot_all_time_series(all_time_series_2019, index, 2019, 'time_series_2019.png')
plot_all_time_series(all_time_series_2020, index, 2020, 'time_series_2020.png')
plot_all_time_series(all_time_series_2021, index, 2021, 'time_series_2021.png')


In [ ]:

## get all index of the time series
index = list(key for key in all_time_series_2021.keys())

# time series arima model
test_series = all_time_series_2021[index[81]]

# add weather feature
weather_2021 = pd.read_csv('../data/weather_data/csv/merged/weather_data_202112.csv')

# add weather feature to time series
test_series = test_series.merge(weather_2021, left_on=['date','hour'], right_on=['date','time'], how='left')
test_series = test_series.drop(columns=['time','date','hour'])

# set datetime as index
test_series['datetime'] = pd.to_datetime(test_series['datetime'])
test_series = test_series.set_index('datetime')

# test_series.to_csv('../data/time_series_0.csv', index=True)



In [ ]:
# use pmdarima to find the best arima model
stepwise_model = auto_arima(test_series['trip_count'], seasonal=True, m=24, 
                   exogenous=test_series[['feels_like','humidity','wind_speed','rain','snow']],
                   suppress_warnings=True, 
                   error_action="ignore", 
                   stepwise=True, 
                   max_order=None,
                   trace=True,
                   suppress_stdout=False,
                   information_criterion='aic',)

stepwise_model.summary()





In [ ]:
# use pmdarima to find the best arima model
stepwise_model1 = auto_arima(np.log(test_series['trip_count']+1), seasonal=True, m=24, 
                   exogenous=test_series[['feels_like','humidity','wind_speed','rain','snow']],
                   suppress_warnings=True, 
                   error_action="ignore", 
                   stepwise=True, 
                   max_order=None,
                   trace=True,
                   suppress_stdout=False,
                   information_criterion='aic',)

stepwise_model1.summary()

In [ ]:
# save the model to disk
filename = '../data/model/arima_model_2019_132to236.sav'
pickle.dump(stepwise_model, open(filename, 'wb'))

In [ ]:
# load the model from disk
filename = '../data/model/arima_model_2017_132to236.sav'
model = pickle.load(open(filename, 'rb'))
model.summary()

In [ ]:
result = stepwise_model1.fit(np.log(test_series['trip_count']+1),exogenous=test_series[['feels_like','humidity','wind_speed','rain','snow']])
print(result.summary())
result.plot_diagnostics(figsize=(15,12))

In [ ]:
result1 = stepwise_model1.fit(np.log(test_series['trip_count']+1),exogenous=test_series[['feels_like','humidity','wind_speed','rain','snow']])
print(result1.summary())
result1.plot_diagnostics(figsize=(15,12))

In [ ]:
stepwise_model.plot_diagnostics(figsize=(15,12))

In [ ]:
stepwise_model1.plot_diagnostics(figsize=(15,12))

In [ ]:
model.plot_diagnostics(figsize=(15,12))

In [ ]:
preds= stepwise_model.fit(test_series['trip_count'], exogenous=test_series[['feels_like','humidity','wind_speed','rain','snow']])

In [ ]:
preds

In [ ]:
model1 = sarimax.SARIMAX(test_series['trip_count'], exog=test_series[['feels_like','humidity','wind_speed','rain','snow']], order=(3, 0, 0), seasonal_order=(2, 0, 0, 24))

model1_fit = model1.fit()

model1_fit.summary()

model1_fit.plot_diagnostics(figsize=(15,12))



In [ ]:
stepwise_model1 = stepwise_model
result = stepwise_model1.fit(test_series['trip_count'])
result.plot_diagnostics(figsize=(15,12))

In [ ]:
# plot a example of how to decompose a time series


decomp = seasonal_decompose(test_series['trip_count'], model='additive',period=168)


trend = decomp.trend
seasonal = decomp.seasonal
residual = decomp.resid

plt.figure(figsize=(15, 8))
plt.subplot(411)
plt.plot(test_series['trip_count'], label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()


